# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [3]:
# load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
os.chdir(r"C:\Users\abewo\Documents\GitHub\GuidedCapstone1\Step Two Data Wrangling\data")

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [4]:
# load csv as df
df = pd.read_csv('step3_output.csv', index_col=0)

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [5]:
# replace state column with dummy variables for state
df = pd.concat([df.drop(labels='state',axis=1), pd.get_dummies(df['state'])], axis=1)

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort which we don't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [6]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X, and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled = scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [7]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
# Call the train_test_split() function with the first two parameters set to X_scaled and y
X_train,X_test,y_train,y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [8]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [9]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [10]:
# You can plug y_test and y_pred into the functions to evaluate the model
print("Explained Variance: " + str(explained_variance_score(y_test, y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, y_pred)))

Explained Variance: 0.6448688396984827
Mean Absolute Error: 7.2639436277763325


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [11]:
print("Lm Intercept: " + str(lm.intercept_))

Lm Intercept: 55.86596960040196


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [12]:
# make a sorted pandas DataFrame displaying the coefficients for each variable
(pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])).sort_values(by=['Coefficient'],ascending=False).head(10)


,Coefficient
trams,1.435983e+12
fastSixes,1.353788e+12
fastEight,8.044484e+11
New York,3.590408e+11
Michigan,3.158509e+11
California,2.851094e+11
Wisconsin,2.737974e+11
Pennsylvania,2.737974e+11
New Hampshire,2.737974e+11
Vermont,2.355957e+11


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [13]:
# second model, without states
df = pd.read_csv('step3_output.csv', index_col=0)
X = df.drop(['Name','AdultWeekend', 'state'], axis=1)
y = df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Explained Variance: " + str(explained_variance_score(y_test, y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, y_pred)))
print("Lm Intercept: " + str(lm.intercept_))

Explained Variance: 0.6682914295901546
Mean Absolute Error: 7.236429253749222
Lm Intercept: 56.01789156941788


In [14]:
# Model Two Top Ten Coefficients
print((pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])).sort_values(by=['Coefficient'],ascending=False).head(10))

                   Coefficient
AdultWeekday          9.809805
clusters              2.939168
summit_elev           2.440356
quad                  1.763540
surface               1.729999
daysOpenLastYear      1.693742
SkiableTerrain_ac     1.677664
vertical_drop         1.522327
projectedDaysOpen     1.266815
fastQuads             1.123641


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [15]:
# third model, without states or summit_elev [note: base_elev was removed in an earlier step of the capstone]
df = pd.read_csv('step3_output.csv', index_col=0)
X = df.drop(['Name','AdultWeekend', 'state', 'summit_elev'], axis=1)
y = df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Explained Variance: " + str(explained_variance_score(y_test, y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, y_pred)))
print("Lm Intercept: " + str(lm.intercept_))

Explained Variance: 0.6380773422196397
Mean Absolute Error: 7.481612884030336
Lm Intercept: 56.06104478290273


In [16]:
# Model Three Top Ten Coefficients
print((pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])).sort_values(by=['Coefficient'],ascending=False).head(10))

                   Coefficient
AdultWeekday          9.681664
clusters              2.521587
SkiableTerrain_ac     2.316150
quad                  1.660349
averageSnowfall       1.574585
daysOpenLastYear      1.494180
surface               1.443356
projectedDaysOpen     1.191774
LongestRun_mi         1.125597
double                1.121385


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -6.57e+23 | 1.69e+12 |-|
| Model 2. | 0.52| 7.53 |'state'|
| Model 3. | 0.52 | 7.52 |'state','summit_elev'|

Model Selection: Model 3